In [7]:
!pip install -U langchain-community sentence-transformers faiss-cpu datasets pandas



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from datasets import load_dataset
import pandas as pd
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaLLM
from langchain.chains import RetrievalQA
import time


In [9]:
print("[*] Завантаження BoolQ...")
ds = load_dataset("boolq")
test_data = ds["train"].shuffle(seed=42).select(range(300))  

questions = [item['question'] for item in test_data]
gold_answers = ["yes" if item['answer'] else "no" for item in test_data]
contexts = [item['passage'] for item in test_data]


[*] Завантаження BoolQ...


In [10]:
print("[*] Векторизація knowledge base...")
embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en")
vectorstore = FAISS.from_texts(contexts, embedder)


[*] Векторизація knowledge base...


In [14]:
from time import time, sleep

print("[*] Запуск baseline (no-RAG)...")
llm = OllamaLLM(model="llama3")

no_rag_answers = []
start_time = time()

for i, q in enumerate(questions):
    try:
        ans = llm.invoke(q)
    except Exception as e:
        ans = f"[Error]: {e}"
    no_rag_answers.append(ans)

    print(f"[{i+1}/{len(questions)}] Q: {q}")
    print(f"  → A: {str(ans)[:100]}{'...' if len(str(ans)) > 100 else ''}\n")


print(f"[*] No-RAG завершено за {time() - start_time:.1f} сек")

[*] Запуск baseline (no-RAG)...
[1/300] Q: did henry die in once upon a time
  → A: The eternal question!

Henry Mills, the protagonist of Once Upon a Time, had a rather complicated st...

[2/300] Q: can i use a tracfone with straight talk service
  → A: The eternal question of wireless freedom!

In short, yes, you can use a TracFone with Straight Talk ...

[3/300] Q: has any nba team ever come back from 3-0 in playoffs
  → A: Yes, one NBA team has come back from a 3-0 deficit in the playoffs: the Golden State Warriors in the...

[4/300] Q: netball can you shoot from outside the circle
  → A: In netball, shooting from outside the goal circle is not allowed. The goal ring is a crucial part of...

[5/300] Q: is the schwarzschild radius the same as the event horizon
  → A: In general relativity, the Schwarzschild radius and the event horizon are related but not exactly th...

[6/300] Q: is shark tank a copy of dragons den
  → A: Shark Tank and Dragons' Den are both reality TV shows where 

In [18]:
from tqdm import tqdm

print("[*] Запуск Retrieval-Augmented Generation (RAG)...")
K = 10
retriever = vectorstore.as_retriever(search_kwargs={"k": K})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

rag_answers = []
start_time = time()  # якщо у тебе from time import time

for i, q in enumerate(tqdm(questions, desc="RAG", unit="Q")):
    try:
        result = qa_chain.invoke(q)
        answer = result["result"]
    except Exception as e:
        answer = f"[Error]: {e}"
    rag_answers.append(answer)

    # Вивід для моніторингу — кожні 5 питань або останнє:
    if i % 5 == 0 or i == len(questions) - 1:
        print(f"[{i+1}/{len(questions)}] Q: {q}")
        print(f"  → A: {str(answer)[:100]}{'...' if len(str(answer)) > 100 else ''}\n")

print(f"[*] RAG завершено за {time() - start_time:.1f} сек")

[*] Запуск Retrieval-Augmented Generation (RAG)...


RAG:   0%|          | 1/300 [00:01<08:58,  1.80s/Q]

[1/300] Q: did henry die in once upon a time
  → A: According to the provided context, yes, Henry Daniel Mills died in Once Upon a Time. He had a brain ...



RAG:   2%|▏         | 6/300 [00:10<09:01,  1.84s/Q]

[6/300] Q: is shark tank a copy of dragons den
  → A: According to the provided context, Shark Tank (originated in the United States) is one of the names ...



...

[291/300] Q: does montag die in fahrenheit 451 the movie
  → A: According to the context provided, Montag does not die in the movie Fahrenheit 451. In fact, it is s...



RAG:  99%|█████████▊| 296/300 [07:37<00:06,  1.71s/Q]

[296/300] Q: is there a remake of the lion king
  → A: Yes, according to the provided context, there is a remake of The Lion King. The article mentions tha...



RAG: 100%|██████████| 300/300 [07:42<00:00,  1.54s/Q]

[300/300] Q: is a tommy gun a sub machine gun
  → A: Yes, according to the provided context, a "Tommy Gun" (also known as the Thompson submachine gun) is...

[*] RAG завершено за 463.0 сек


In [19]:
def recall_at_k(retriever, questions, gt_contexts, k):
    match_count = 0
    for i, q in enumerate(questions):
        retrieved = retriever.get_relevant_documents(q)
        if any(gt_contexts[i].strip() == doc.page_content.strip() for doc in retrieved):
            match_count += 1
    return match_count / len(questions)

recall = recall_at_k(retriever, questions, contexts, K)
print(f"[*] Recall@{K}: {recall:.3f}")


C:\Users\user\AppData\Local\Temp\ipykernel_14068\281558299.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved = retriever.get_relevant_documents(q)


[*] Recall@10: 0.997


In [20]:
def extract_yes_no(answer: str):
    answer = answer.lower()
    if "yes" in answer: return "yes"
    if "no" in answer: return "no"
    return "unknown"

df = pd.DataFrame({
    "question": questions,
    "context": contexts,
    "gold_answer": gold_answers,
    "no_rag_answer": no_rag_answers,
    "rag_answer": rag_answers
})
df["no_rag_pred"] = df["no_rag_answer"].apply(extract_yes_no)
df["rag_pred"] = df["rag_answer"].apply(extract_yes_no)

acc_norag = (df["no_rag_pred"] == df["gold_answer"]).mean()
acc_rag = (df["rag_pred"] == df["gold_answer"]).mean()
print(f"[*] No-RAG accuracy: {acc_norag:.3f}")
print(f"[*] RAG accuracy:   {acc_rag:.3f}")

df.to_csv("boolq_rag_results.csv", index=False)
print("[*] Збережено у boolq_rag_results.csv")


[*] No-RAG accuracy: 0.517
[*] RAG accuracy:   0.720
[*] Збережено у boolq_rag_results.csv
